In [1]:
import numpy as np
import scipy.io as sio
from scipy.linalg import toeplitz
from scipy.stats import multivariate_normal
import time

from L0Obj import L0Obj
from ProjCSimplex import ProjCSimplex
from minConf.minConf_PQN import minConF_PQN
import random

# Functions

## Functions to generate data

In [78]:
def add_noise_to_signal(signal, desired_snr_db):
    # Convert signal to numpy array
    signal = np.asarray(signal)
    
    # Calculate signal power
    signal_power = np.mean(signal ** 2)
    
    # Calculate the desired noise power based on the desired SNR (in dB)
    desired_snr_linear = 10 ** (desired_snr_db / 10)
    noise_power = signal_power / desired_snr_linear
    
    # Generate noise with the calculated power
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    
    # Add noise to the original signal
    noisy_signal = signal + noise
    
    return noisy_signal, noise


def compute_snr(signal, noise):
    # Convert signal and noise to numpy arrays for calculations
    signal = np.asarray(signal)
    noise = np.asarray(noise)

    # Calculate the power of the signal and the noise
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    
    # Calculate the SNR
    snr = signal_power / noise_power
    
    # Convert SNR to decibels (dB)
    snr_db = 10 * np.log10(snr)
    
    return snr_db


def SynData_iid(n, p, b, snr):
    #n: the number of samples
    #p: the number of features
    #b: the number of groups
    #snr: signal-noise-ratio snr = 10*log_10 E[S^2]/E[N^2] 
    
    # hard coding sparsity setting
    Sparse = [10,10,10,10,10]#[8, 6, 4, 2, 1]
    Group = {}
    for i in range(len(Sparse)):
        Group[i] = 100*i + np.random.permutation(int(p/b))[0:Sparse[i]]
    print(Group)

    Mean = np.zeros(p)
    Cov = np.eye(p)
    
    X = np.random.multivariate_normal(Mean, Cov, n)
    print(X.shape)
    w = np.zeros(p)
    for i in range(len(Group)):
        RandBin = 2*np.random.randint(0, 2, size=len(Group[i]))-1
        w[Group[i]] = RandBin
    w = w[:,np.newaxis]

    Signal = X.dot(w)

    Noisy_Signal, Noise = add_noise_to_signal(Signal, snr)

    SNR_Cmp = compute_snr(Signal, Noise)
    print("Computed SNR:", SNR_Cmp)

    y = Noisy_Signal
    
    # groups info for L0_GL
    gn = b
    Group = []
    for i in range(1, b + 1):
        Group.append(list(range((i-1)*int(p/b) , i*int(p/b))))

    return y, X, w, Group

def Data4Matlab(X,w,y):
    d = len(w)
    n = len(y)
    # for matlab
    Folder = "./RandomEnsembleI_Known_k_h/"
    fn1 = Folder + "Xnd.txt"
    fh1 = open(fn1,'w')
    for i in range(n):
        for j in range(d-1):
            fh1.write("%f\t" % (X[i,j]))
        fh1.write("%f\n" % (X[i,d-1]))
    fh1.close()

    fn2 = Folder + "yn.txt"
    fh2 = open(fn2, 'w')
    for i in range(n):
        fh2.write("%f\n"%(y[i]))
    fh2.close()

    fn3 = Folder + "wd.txt"
    fh3 = open(fn3, 'w')
    for i in range(d):
        fh3.write("%f\n"%(w[i]))
    fh3.close()


## Functions of the algorithms

In [30]:
import numpy as np
from scipy.sparse import spdiags
from scipy.linalg import inv
from gurobipy import Model, GRB, QuadExpr

def L0GLObj(u, X, y, pho):
    # u (feature, 1)
    # X (instance, feature)
    # y (instance, 1)
    # pho (1, 1)
    #print(pho)
    #print(u)
    n, d = X.shape
    SpDiag = spdiags(u.flatten(), 0, d, d)
    # print(SpDiag)
    # print(f"SpDiag: {SpDiag.toarray()[-1][-1]}")
    M = inv((1/pho) * X @ SpDiag @ X.conj().T + np.eye(n))
    f = y.conj().T @ M @ y

    g = -(1/pho) * ((X.conj().T @ M @ y)**2)

    return f, g

def ProjCSimplexGL_Gurobi(u, k, Group, h):
    n, m = u.shape
    g = len(Group)
    
    # Create matrices H1 and H2
    H1 = np.eye(n)
    f1 = -2*u.flatten()

    H2 = np.zeros((g, g))
    f2 = np.zeros(g)

    H = np.block([[H1, np.zeros((n, g))], [np.zeros((g, n)), H2]])
    f = np.concatenate([f1, f2])
    #print(u.shape)
    #print(f.shape)

    # Constructing constraint matrices A1 and A2
    A1 = np.ones((1, n))
    A2 = np.zeros((1, g))
    gcn = 0

    for i in range(g):
        for j in range(len(Group[i])):
            A1t = np.zeros((1, n))
            A1t[0, Group[i][j]] = 1
            A1 = np.vstack([A1, A1t])
            
            A2t = np.zeros((1, g))
            A2t[0, i] = -1
            A2 = np.vstack([A2, A2t])
            gcn += 1

    A1 = np.vstack([A1, np.zeros((1, n))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])

    # RHS vector
    b = np.zeros(gcn + 2)
    b[0] = k
    b[-1] = h

    # Lower and upper bounds
    lb = np.zeros(n + g)
    ub = np.ones(n + g)

    # Setup Gurobi model
    model = Model()

    # Add variables
    Q = H
    x = model.addMVar(n + g, ub=1.0, lb=0.0)
    
    model.setObjective(x@Q@x+ x@f)
    model.addConstr(A @ x <= b)
    

    # Set Gurobi parameters
    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 5)

    # Optimize model
    model.optimize()

    # Get the results
    up = x.x[:n]
    zp = x.x[n:]

    #print(up.shape)
    return up[:,np.newaxis]

# Experiment

In [79]:
# Parameters for generating data
n=500            # the number of samples
p=1000          # the number of features
b=10            # the number of groups
snr = 4        # signal-noise-ratio

# Parameters for the algorithm
k=50
h=5
pho = np.sqrt(n)

In [80]:
Resp, FeatureM, w, Group =  SynData_iid(n, p, b, snr)
Data4Matlab(FeatureM,w,Resp)

{0: array([59, 47, 34, 99, 74,  2, 81, 36, 51, 65]), 1: array([117, 138, 112, 177, 115, 125, 163, 114, 109, 123]), 2: array([292, 238, 258, 203, 242, 223, 209, 246, 296, 268]), 3: array([393, 387, 307, 347, 383, 316, 386, 337, 313, 374]), 4: array([466, 460, 454, 487, 480, 499, 428, 431, 409, 465])}
(500, 1000)
Computed SNR: 4.501181995282714


FileNotFoundError: [Errno 2] No such file or directory: './RandomEnsembleI_Known_k_h/Xnd.txt'

In [73]:
uSimplex = np.ones((p, 1)) * (1 / p)
X = FeatureM
y = Resp

In [74]:
# Set up Objective Function
funObj = lambda w: L0GLObj(w, X, y, pho)

# Set up Simplex Projection Function
funProj = lambda w: ProjCSimplexGL_Gurobi(w, k, Group, h)

In [75]:
options = {'maxIter': 50}
tStart = time.process_time()
uout, obj, _ = minConF_PQN(funObj, uSimplex, funProj, options)
# data = sio.loadmat('./var_data_NIPS_end.mat')
# uout, obj, _ = data["uout"], data["obj"], data["a"]
#print(f"uout: {uout}")
tEnd = time.process_time() - tStart

 Iteration   FunEvals Projections     Step Length    Function Val        Opt Cond
         1          2          4     5.23621e-07     3.59056e+04     5.09000e+01
         2          3         16     1.00000e+00     2.79748e+04     5.08529e+01
         3          4         28     1.00000e+00     2.34661e+04     5.12431e+01
         4          5         40     1.00000e+00     1.85972e+04     5.22833e+01
         5          6         52     1.00000e+00     1.48352e+04     5.42274e+01
         6          7         64     1.00000e+00     1.16316e+04     5.76998e+01
         7          8         76     1.00000e+00     9.04546e+03     6.34660e+01
         8          9         88     1.00000e+00     6.96117e+03     7.22494e+01
         9         10        100     1.00000e+00     5.91832e+03     7.39409e+01
        10         11        112     1.00000e+00     5.51633e+03     6.80454e+01
        11         12        124     1.00000e+00     5.32741e+03     7.56998e+01
        12         13      

In [76]:
uout = ProjCSimplexGL_Gurobi(uout, k, Group, h)
B = np.sort(-uout.flatten())
Ranktmp = np.argsort(-uout.flatten())
Rank = np.sort(Ranktmp[:k])

utrue = w
Indtrue = np.where(utrue)
C = np.intersect1d(Rank, Indtrue)
AccPQN = len(C) / k

print(AccPQN)

0.8


In [77]:
uout[0:20], w[0:20]

(array([[0.        ],
        [0.08349885],
        [0.29930249],
        [0.        ],
        [0.        ],
        [0.02876783],
        [0.        ],
        [0.        ],
        [0.38235069],
        [0.        ],
        [0.20082421],
        [0.09831413],
        [0.109971  ],
        [0.        ],
        [0.03548627],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.06569697],
        [0.        ]]),
 array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]))

In [61]:
X.shape

(40, 1000)

In [8]:
cov = np.array([[6, -3], [-3, 3.5]])
pts = np.random.multivariate_normal([0, 0], cov, size=10000)

In [16]:
pts.shape

(10000, 2)

In [11]:
np.mean(pts[:,0]), np.var(pts[:,0])

(-0.01834594430993853, 5.828645479309693)

In [12]:
np.mean(pts[:,1]), np.var(pts[:,1])

(-0.012953112568373098, 3.4267962342808373)

In [14]:
pts.T

array([[-2.01175595, -2.97146764, -2.44843331, ..., -2.39147332,
        -6.28507677, -1.48392745],
       [ 1.77173421,  1.80522279,  1.51965345, ...,  0.99018067,
         1.38301169,  2.44753503]])

In [15]:
np.cov(pts.T)

array([[ 5.8292284 , -2.91685259],
       [-2.91685259,  3.42713895]])